In [1]:
import string
import re
from datetime import datetime
import locale
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [2]:
pd.set_option('display.max_colwidth', 500)

# Global Functions and Variable

In [2]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

stop_words = set(stopwords.words('indonesian'))
exceptions = {"sampai", "kurang", "lebih", "dari", "hari"}
custom_stop_words = stop_words - exceptions

In [3]:
istilah_kesehatan = {
    # Tekanan Darah
    "hipertensi": "tekanan darah tinggi",
    "hipotensi": "tekanan darah rendah",

    # Gula Darah
    "hiperglikemia": "kadar gula darah tinggi",
    "hipoglikemia": "kadar gula darah rendah",
    "diabetes melitus": "penyakit kencing manis",

    # Pernapasan
    "dispnea": "sesak napas",
    "rinorea": "pilek atau ingusan",
    "epistaksis": "mimisan",
    "faringitis": "radang tenggorokan",
    "laringitis": "radang pita suara",
    "asma": "bengek",

    # Pencernaan
    "dispepsia": "mag atau gangguan pencernaan",
    "konstipasi": "sembelit atau susah buang air besar",
    "diare": "mencret",
    "hemoroid": "wasir atau ambeien",
    "apendisitis": "radang usus buntu",
    "gastritis": "radang lambung",

    # Kulit dan Alergi
    "urtikaria": "biduran atau kaligata",
    "dermatitis": "eksim atau radang kulit",
    "varisela": "cacar air",
    "morbili": "campak atau tampek",
    "veruka": "kutil",
    "tinea pedis": "kutu air",
    "lotion": "losion",

    # Kepala dan Saraf
    "sefalgia": "sakit kepala atau pusing",
    "insomnia": "susah tidur",
    "sinkop": "pingsan",
    "konvulsi": "kejang",

    # Umum
    "pireksia": "demam",
    "hipertermia": "suhu tubuh sangat tinggi",
    "hipotermia": "suhu tubuh sangat rendah",
    "mialgia": "nyeri otot",
    "artralgia": "nyeri sendi",
    "fatik": "kelelahan atau rasa capai",
    "edema": "bengkak",
    "pruritus": "gatal-gatal",
    "anemia": "kurang darah",
    "karsinoma": "kanker",
    "neoplasma": "tumor",
    "halitosis": "bau mulut",
    "kalkulus renal": "batu ginjal",
    "moisturizing": "moisturizer"
}

In [4]:
# Mapping bulan
bulan = {
    # Nama singkat
    "Jan": "Januari",
    "Feb": "Februari",
    "Mar": "Maret",
    "Apr": "April",
    "Mei": "Mei",
    "Jun": "Juni",
    "Jul": "Juli",
    "Agu": "Agustus",
    "Sep": "September",
    "Okt": "Oktober",
    "Nov": "November",
    "Des": "Desember",

    # Angka
    "01": "Januari",
    "02": "Februari",
    "03": "Maret",
    "04": "April",
    "05": "Mei",
    "06": "Juni",
    "07": "Juli",
    "08": "Agustus",
    "09": "September",
    "10": "Oktober",
    "11": "November",
    "12": "Desember",
}

In [5]:
# Fungsi untuk stemming
def stem_tokens(tokens):
    lemmas = [stemmer.stem(token) for token in tokens]
    return lemmas

In [6]:
def normalize_text(text, dictionary):
    text = re.sub(r'(\d+)\s*[–-]\s*(\d+)', r'\1 sampai \2', text)
    text = text.replace('>', ' lebih dari ')
    text = text.replace('<', ' kurang dari ')
    
    tokens = text.split()

    normalized_tokens = []
    for token in tokens:
        normalized_token = dictionary.get(token, token)
        normalized_tokens.append(normalized_token)

    return " ".join(normalized_tokens)

In [7]:
def preprocess_title_desc(text):
    text = text.lower()
    text = normalize_text(text, istilah_kesehatan)
    text = re.sub(r'\s+', ' ', text) # Remove extra whitespace
    text = re.sub(r'[^\w\s]', '', text) # Remove non-words and non-white spaces
    text = ''.join([char for char in text if char not in string.punctuation]) # Remove punctuations
    text = ' '.join([word for word in text.split() if word not in custom_stop_words]) # Stopwords removal
    text = ' '.join(stem_tokens(text.split()))

    return text

# Biofarma

In [8]:
biofarma_df = pd.read_csv('../artikel/biofarma.csv')
biofarma_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   title        108 non-null    object 
 1   tag          0 non-null      float64
 2   link         108 non-null    object 
 3   date         108 non-null    object 
 4   description  108 non-null    object 
 5   content      108 non-null    object 
dtypes: float64(1), object(5)
memory usage: 5.2+ KB


In [9]:
biofarma_df['source'] = 'Biofarma'
biofarma_df.head()

,title,tag,link,date,description,content,source
0,5 Cara Merawat Ginjal agar Sehat & Cegah Penya...,NaN,https://www.biofarma.co.id/id/announcement/det...,"Rabu, 3 September 2025",Kenali cara merawat ginjal agar terhindar dari...,Ginjal adalah organ vital yang berfungsi menya...,Biofarma
1,7 Tips Untuk Menjaga Kesehatan Mata,NaN,https://www.biofarma.co.id/id/announcement/det...,"Jumat, 29 Agustus 2025","Tips menjaga kesehatan mata: nutrisi bergizi, ...",Menjaga kesehatan mata sangatlah penting karen...,Biofarma
2,Jangan Sepelekan! 10 Penyebab Campak yang Seri...,NaN,https://www.biofarma.co.id/id/announcement/det...,"Kamis, 28 Agustus 2025","Kenali faktor utama penyebab campak, gejala kh...",Penyebab Campak\nCampak adalah penyakit menula...,Biofarma
3,"Campak - Penyebab, Gejala, dan Pencegahannya",NaN,https://www.biofarma.co.id/id/announcement/det...,"Senin, 25 Agustus 2025",Campak bisa berbahaya bila dibiarkan. Periksa ...,Apa itu campak?\nCampak atau rubella adalah pe...,Biofarma
4,Cara Mudah Menjaga Kesehatan Gigi dan Mulut ag...,NaN,https://www.biofarma.co.id/id/announcement/det...,"Rabu, 20 Agustus 2025",Kesehatan gigi dan mulut adalah kondisi ketika...,Apa itu Kesehatan Gigi dan Mulut?\nKesehatan g...,Biofarma


In [10]:
biofarma_df.isnull().sum()

title            0
tag            108
link             0
date             0
description      0
content          0
source           0
dtype: int64

In [11]:
biofarma_df['tag'] = biofarma_df['tag'].fillna('Kesehatan Umum')

In [12]:
biofarma_df['title'].duplicated().sum()

np.int64(0)

In [13]:
def preprocess_date_biofarma(text):
    text = re.sub(r'^\s*\w+,\s*', '', text)
    return text.strip()

In [14]:
def preprocess_content_biofarma(text):
    text = text.lower()
    text = normalize_text(text, istilah_kesehatan)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE) # Remove URLs
    text = re.sub(r'\@w+|\#','', text) # Remove mentions and hashtags
    text = re.sub(r'\s*\d{1,2}[/-]\d{1,2}(?:[/-]\d{2,4})?\s*', 'DATE', text) # Remove dates in parentheses
    text = re.sub(r'\s+', ' ', text) # Remove extra whitespace
    text = re.sub(r'[^\w\s]', '', text) # Remove non-words and non-white spaces
    text = re.sub(r'referensi[\s\S]*?(bagikan:|download|$)', '', text, flags=re.IGNORECASE) # Remove blok referensi ke akhir
    text = ''.join([char for char in text if char not in string.punctuation]) # Remove punctuations
    text = ' '.join([word for word in text.split() if word not in custom_stop_words]) # Stopwords removal
    text = ' '.join(stem_tokens(text.split()))

    return text

In [15]:
biofarma_df['title'] = biofarma_df['title'].apply(lambda x: preprocess_title_desc(x))
biofarma_df['description'] = biofarma_df['description'].apply(lambda x: preprocess_title_desc(x))
biofarma_df['content'] = biofarma_df['content'].apply(lambda x: preprocess_content_biofarma(x))
biofarma_df['date'] = biofarma_df['date'].apply(lambda x: preprocess_date_biofarma(x))

In [16]:
biofarma_df.head()

,title,tag,link,date,description,content,source
0,5 rawat ginjal sehat cegah sakit ginjal,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/5-cara-merawat-ginjal-agar-sehat-cegah-penyakit-ginjal,3 September 2025,kenal rawat ginjal hindar dari gejala ginjal masalah ajar ciri sakit ginjal cegah sakit ginjal,ginjal organ vital fungsi nyaring darah buang limbah jaga imbang cair tubuh jaga fungsi ginjal turun timbul serius sakit ginjal kembang gejala tahap jaga sehat ginjal 5 rawat ginjal sederhana efektif cegah gejala ginjal masalah 1 minum air putih asupan cair bantu ginjal optimal buang racun sisa metabolisme kurang minum air sebab dehidrasi tingkat risiko batu ginjal biasa minum 8 sampai 10 gelas air putih hari jaga fungsi ginjal sehat 2 batas konsumsi garam asupan garam lebih tingkat tekan da...,Biofarma
1,7 tips jaga sehat mata,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/7-tips-untuk-menjaga-kesehatan-mata,29 Agustus 2025,tips jaga sehat mata nutrisi gizi kacamata lindung olahraga periksa rutin dokter,jaga sehat mata mata peran aktivitas seharihari dari ajar interaksi lingkung sehat mata abai muncul dari iritasi ringan infeksi sakit serius risiko sebab buta jaga mata periksa rutin dokter biasa pola hidup sehat dukung fungsi lihat optimal iring tambah usia biasa seharihari kurang sehat fungsi mata turun cegah langkah sederhana jaga asupan nutrisi istirahat tatap layar lindung mata dari papar sinar lebih jaga kualitas lihat tahan lebih tips jaga sehat mata mata salah organ aktif hari lihat ...,Biofarma
2,sepele 10 sebab campak abai,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/jangan-sepelekan-10-penyebab-campak-yang-sering-terabaikan,28 Agustus 2025,kenal faktor utama sebab campak gejala khas cegah imunisasi mmr,sebab campak campak sakit tular sebab virus campak tampek sebar cepat dari orang orang mekanisme utama tular percik air liur droplet dari mulut hidung derita batuk bersin bicara tular sentuh benda kontaminasi virus sentuh hidung mulut mata puluh faktor sebab pengaruh vaksinasi campak mmr campak serang individu vaksin vaksin mmr measles mumps rubella bukti efektif cegah sakit anakanak terima vaksin lengkap milik risiko lebih infeksi milik kebal tubuh virus campak tampek virus usia bayi muda v...,Biofarma
3,campak sebab gejala cegah,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/campak-penyebab-gejala-dan-pencegahannya,25 Agustus 2025,campak bahaya biar periksa dokter gejala muncul tangan lebih,campak campak rubella sakit tular sebab virus dari keluarga paramyxoviridae genus morbillivirus sakit serang salur napas timbul gejala sistemik demam batuk pilek sakit tenggorok mata merah ruam kulit campak sakit tular pindah dari orang orang udara percik air liur benda kontaminasi virus gejala campak gejala campak muncul inkubasi 7 sampai 14 hari gejala flu demam lemas batuk kering pilek sakit tenggorok diare muntah mata merah air sensitif cahaya bercak putih bintik koplik muncul mulut hari...,Biofarma
4,mudah jaga sehat gigi mulut hindar dari sakit,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/cara-mudah-menjaga-kesehatan-gigi-dan-mulut-agar-terhindar-dari-penyakit,20 Agustus 2025,sehat gigi mulut kondisi gigi gusi lidah rongga mulut jaga bersih fungsi mulut gerbang utama makan minum masuk tubuh organ peran,sehat gigi mulut sehat gigi mulut kondisi gigi gusi lidah rongga mulut jaga bersih fungsi mulut gerbang utama makan minum masuk tubuh organ peran proses cerna makan masuk tinggal mulut bersih sisa kembang bakteri sehat mulut kait indah senyum napas segar tentu sehat tubuh akibat jaga sehat gigi mulut abai bersih mulut gigi picu ganggu sehat muncul bau mulut halitosis sisa makan busuk mulut akibat biasa rokok gigi lubang akibat tumpuk plak kikis email gigi radang gusi gingivitis atas kembang ...,Biofarma


# Klikdokter

In [16]:
klikdokter_df = pd.read_csv('../artikel/klikdokter.csv')
klikdokter_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        110 non-null    object
 1   tag          110 non-null    object
 2   link         110 non-null    object
 3   date         110 non-null    object
 4   description  103 non-null    object
 5   content      110 non-null    object
dtypes: object(6)
memory usage: 5.3+ KB


In [17]:
klikdokter_df['source'] = 'Klikdokter'
klikdokter_df.head()

,title,tag,link,date,description,content,source
0,Jangan Panik! Kenali Rabies dan Pertolongan Pe...,Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-k...,30 Mei 2025,NaN,Berita Kesehatan\nJangan Panik! Kenali Rabies ...,Klikdokter
1,Terapi Stem Cell untuk Pengobatan Penyakit Aut...,Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-k...,29 Apr 2025,Penyakit autoimun seperti rheumatoid arthritis...,Berita Kesehatan\nTerapi Stem Cell untuk Pengo...,Klikdokter
2,"Terapi Stem Cell, Harapan Baru untuk Pengobata...",Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-k...,29 Apr 2025,Terapi stem cell kini menjadi harapan baru dal...,"Berita Kesehatan\nTerapi Stem Cell, Harapan Ba...",Klikdokter
3,"Stem Cell Tumbuhan dan Manusia Itu Berbeda, In...",Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-k...,19 Apr 2025,Baik tumbuhan maupun manusia memiliki stem cel...,Berita Kesehatan\nStem Cell Tumbuhan dan Manus...,Klikdokter
4,Keberagaman Model di Victoria’s Secret Fashion...,Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-k...,17 Okt 2024,Victoria's Secret ubah standar kecantikan! Sho...,Berita Kesehatan\nKeberagaman Model di Victori...,Klikdokter


In [19]:
klikdokter_df.isnull().sum()

title          0
tag            0
link           0
date           0
description    7
content        0
source         0
dtype: int64

In [18]:
klikdokter_df['title'].duplicated().sum()

np.int64(0)

In [19]:
klikdokter_df['description'] = klikdokter_df['description'].fillna('Tidak ada deskripsi')

In [20]:
def preprocess_date_klikdokter(text, dictionary):    
    tokens = text.split()

    normalized_tokens = []
    for token in tokens:
        normalized_token = dictionary.get(token, token)
        normalized_tokens.append(normalized_token)

    return " ".join(normalized_tokens)

In [21]:
def preprocess_content_klikdokter(text):
    text = text.lower()
    text = normalize_text(text, istilah_kesehatan)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE) # Remove URLs
    text = re.sub(r'\@w+|\#','', text) # Remove mentions and hashtags
    text = re.sub(r'^.*?Bagikan', '', text, flags=re.DOTALL|re.IGNORECASE) # Remove pendahuluan artikel
    text = re.sub(r'advertisement', '', text, flags=re.IGNORECASE) # Remove advertisement
    text = re.sub(r'^artikel lainnya:.*', '', text, flags=re.MULTILINE) # Remove artikel lainnya
    text = re.sub(r'Yuk, download.*?Referensi', '', text, flags=re.DOTALL|re.IGNORECASE) # Remove penutup artikel
    text = re.sub(r'Referensi.*', '', text, flags=re.IGNORECASE) # Remove referensi
    text = re.sub(r'\s*\d{1,2}[/-]\d{1,2}(?:[/-]\d{2,4})?\s*', 'DATE', text) # Remove dates in parentheses
    text = re.sub(r'\s+', ' ', text) # Remove extra whitespace
    text = re.sub(r'[^\w\s]', '', text) # Remove non-words and non-white spaces
    text = ''.join([char for char in text if char not in string.punctuation]) # Remove punctuations
    text = ' '.join([word for word in text.split() if word not in custom_stop_words]) # Stopwords removal
    text = ' '.join(stem_tokens(text.split()))

    return text

In [22]:
klikdokter_df['title'] = klikdokter_df['title'].apply(lambda x: preprocess_title_desc(x))
klikdokter_df['description'] = klikdokter_df['description'].apply(lambda x: preprocess_title_desc(x))
klikdokter_df['content'] = klikdokter_df['content'].apply(lambda x: preprocess_content_klikdokter(x))
klikdokter_df['date'] = klikdokter_df['date'].apply(lambda x: preprocess_date_klikdokter(x, bulan))

In [23]:
klikdokter_df.head()

,title,tag,link,date,description,content,source
0,panik kenal rabies tolong gigit hewan liar,Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-k...,30 Mei 2025,deskripsi,tahu rabies bahaya ancam nyawa sakit serang si...,Klikdokter
1,terapi stem cell obat sakit autoimun,Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-k...,29 April 2025,sakit autoimun rheumatoid arthritis turun kual...,sakit autoimun rheumatoid arthritis turun kual...,Klikdokter
2,terapi stem cell harap obat sakit degeneratif,Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-k...,29 April 2025,terapi stem cell harap tangan sakit degenerati...,terapi stem cell harap tangan sakit degenerati...,Klikdokter
3,stem cell tumbuh manusia beda jelas,Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-k...,19 April 2025,tumbuh manusia milik stem cell nnamun milik be...,tumbuh manusia milik stem cell nnamun milik be...,Klikdokter
4,agam model victorias secret fashion show kenal...,Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-k...,17 Oktober 2024,victorias secret ubah standar cantik show baru...,victorias secret ubah standar cantik show baru...,Klikdokter


# Halodoc

In [24]:
halodoc_df = pd.read_csv('../artikel/halodoc.csv')
halodoc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        72 non-null     object
 1   tag          72 non-null     object
 2   link         72 non-null     object
 3   date         69 non-null     object
 4   description  72 non-null     object
 5   content      69 non-null     object
dtypes: object(6)
memory usage: 3.5+ KB


In [25]:
halodoc_df['source'] = 'halodoc'

In [26]:
halodoc_df.isnull().sum()

title          0
tag            0
link           0
date           3
description    0
content        3
source         0
dtype: int64

In [28]:
halodoc_df['title'].duplicated().sum()

np.int64(0)

In [27]:
halodoc_df.dropna(inplace=True)

In [29]:
halodoc_df.head()

,title,tag,link,date,description,content,source
0,Kenalan dengan Manfaat Ceramide untuk Perawata...,"Perawatan Kulit, Kecantikan",https://www.halodoc.com/artikel/kenalan-dengan...,15 September 2025,Ceramide mampu melindungi skin barrier secara ...,Advertisement\nTemukan lebih banyak\nHalodoc\n...,halodoc
1,Sakit Perut Sebelah Kanan Bawah Sampai ke Ping...,Sakit Pinggang,https://www.halodoc.com/artikel/sakit-perut-se...,15 September 2025,Sakit perut sebelah kanan bawah sampai ke ping...,Advertisement\nSakit Perut Sebelah Kanan Bawah...,halodoc
2,"Mual dan Muntah, Kenali Gejala Alami Infeksi A...","Demam, Astrovirus",https://www.halodoc.com/artikel/mual-dan-munta...,15 September 2025,Infeksi astrovirus dapat menyebabkan gastroent...,"Advertisement\nMual dan Muntah, Kenali Gejala ...",halodoc
3,Apa Saja Kandungan Vitamin yang Terdapat dalam...,Makanan Sehat,https://www.halodoc.com/artikel/apa-saja-kandu...,15 September 2025,"Buah salak mengandung nutrisi vitamin A, B, C,...",Advertisement\nApa Saja Kandungan Vitamin yang...,halodoc
4,Ketahui Fungsi Rongga Mulut dalam Sistem Pence...,Mulut dan Gigi,https://www.halodoc.com/artikel/ketahui-fungsi...,15 September 2025,"Rongga mulut berfungsi untuk mengunyah, mempro...",Advertisement\nKetahui Fungsi Rongga Mulut dal...,halodoc


In [30]:
def preprocess_content_halodoc(text):
    text = text.lower()
    text = normalize_text(text, istilah_kesehatan)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE) # Remove URLs
    text = re.sub(r'\@w+|\#','', text) # Remove mentions and hashtags
    text = re.sub(r'advertisement', '', text, flags=re.IGNORECASE) # Remove advertisement
    text = re.sub(r'^.*?daftar isi', '', text, flags=re.DOTALL|re.IGNORECASE)
    text = re.sub(r'mau tahu.*?referensi', '', text, flags=re.DOTALL|re.IGNORECASE)
    text = re.sub(r'referensi.*$', '', text, flags=re.DOTALL|re.IGNORECASE)
    text = re.sub(r'\s*\d{1,2}[/-]\d{1,2}(?:[/-]\d{2,4})?\s*', 'DATE', text) # Remove dates in parentheses
    text = re.sub(r'\s+', ' ', text) # Remove extra whitespace
    text = re.sub(r'[^\w\s]', '', text) # Remove non-words and non-white spaces
    text = ''.join([char for char in text if char not in string.punctuation]) # Remove punctuations
    text = ' '.join([word for word in text.split() if word not in custom_stop_words]) # Stopwords removal
    text = ' '.join(stem_tokens(text.split()))

    return text

In [31]:
halodoc_df['title'] = halodoc_df['title'].apply(lambda x: preprocess_title_desc(x))
halodoc_df['description'] = halodoc_df['description'].apply(lambda x: preprocess_title_desc(x))
halodoc_df['content'] = halodoc_df['content'].apply(lambda x: preprocess_content_halodoc(x))

In [32]:
halodoc_df.head()

,title,tag,link,date,description,content,source
0,kenal manfaat ceramide awat kulit,"Perawatan Kulit, Kecantikan",https://www.halodoc.com/artikel/kenalan-dengan...,15 September 2025,ceramide lindung skin barrier efektif,manfaat ceramide kulit rekomendasi skincare ka...,halodoc
1,sakit perut belah kanan sampai pinggang sebab ...,Sakit Pinggang,https://www.halodoc.com/artikel/sakit-perut-se...,15 September 2025,sakit perut belah kanan sampai pinggang tanda ...,sebab sakit perut belah kanan sampai pinggang ...,halodoc
2,mual muntah kenal gejala alami infeksi astrovirus,"Demam, Astrovirus",https://www.halodoc.com/artikel/mual-dan-munta...,15 September 2025,infeksi astrovirus sebab gastroenteritis gejal...,gejala infeksi astrovirus muntah gejala infeks...,halodoc
3,kandung vitamin buah salak,Makanan Sehat,https://www.halodoc.com/artikel/apa-saja-kandu...,15 September 2025,buah salak kandung nutrisi vitamin a b c zat b...,kandung vitamin nutrisi buah salak manfaat sal...,halodoc
4,tahu fungsi rongga mulut sistem cerna manusia,Mulut dan Gigi,https://www.halodoc.com/artikel/ketahui-fungsi...,15 September 2025,rongga mulut fungsi kunyah proses makan kimiaw...,fungsi utama rongga mulut anatomi fungsi mulut...,halodoc


# Alodokter

In [38]:
alodokter_df = pd.read_csv('../artikel/alodokter.csv')
alodokter_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        100 non-null    object
 1   tag          100 non-null    object
 2   link         100 non-null    object
 3   date         100 non-null    object
 4   description  100 non-null    object
 5   content      100 non-null    object
dtypes: object(6)
memory usage: 4.8+ KB


In [39]:
alodokter_df['source'] = 'alodokter'
alodokter_df.head()

,title,tag,link,date,description,content,source
0,"Overthinking, Kenali Ciri, Penyebab, Dampak, d...",HIDUP SEHAT,https://www.alodokter.com/overthinking-kenali-...,Terakhir diperbarui: 16 September 2025,Overthinking adalah kebiasaan memikirkan sesua...,Overthinking adalah kebiasaan memikirkan sesua...,alodokter
1,"Trauma Healing, Cara Membantu Diri Pulih dari ...",KESEHATAN,https://www.alodokter.com/trauma-healing-cara-...,Terakhir diperbarui: 16 September 2025,Trauma healing adalah proses penting untuk mem...,Trauma healing adalah proses penting untuk mem...,alodokter
2,"Mansplaining, Perilaku Merendahkan yang Sering...",KESEHATAN,https://www.alodokter.com/mansplaining-perilak...,Terakhir diperbarui: 16 September 2025,Mansplaining adalah pola komunikasi ketika ses...,Mansplaining adalah pola komunikasi ketika ses...,alodokter
3,"Fungsi Otak Kiri, Kunci Kemampuan Analitis dan...",KESEHATAN,https://www.alodokter.com/fungsi-otak-kiri-kun...,Terakhir diperbarui: 16 September 2025,Fungsi otak kiri memegang peranan penting dala...,Fungsi otak kiri memegang peranan penting dala...,alodokter
4,"Daun Jati Belanda, Kenali Manfaatnya untuk Kes...",KESEHATAN,https://www.alodokter.com/daun-jati-belanda-ke...,Terakhir diperbarui: 16 September 2025,Daun jati belanda merupakan herbal tradisional...,Daun jati belanda merupakan herbal tradisional...,alodokter


In [40]:
alodokter_df['title'].duplicated().sum()

np.int64(34)

In [41]:
alodokter_df.drop_duplicates(subset='title', inplace=True)
alodokter_df['title'].duplicated().sum()

np.int64(0)

In [42]:
def preprocess_content_alodokter(text):
    text = str(text).lower()
    text = normalize_text(text, istilah_kesehatan)  # Normalisasi istilah kesehatan
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # Hapus URL
    text = re.sub(r'\@w+|\#', '', text)  # Hapus mention & hashtag
    text = re.sub(r'advertisement', '', text, flags=re.IGNORECASE)  # Hapus "advertisement"
    text = re.sub(r'^.*?daftar isi', '', text, flags=re.DOTALL|re.IGNORECASE)  # Hapus "daftar isi"
    text = re.sub(r'referensi.*$', '', text, flags=re.DOTALL|re.IGNORECASE)  # Hapus "referensi"
    text = re.sub(r'\s*\d{1,2}[/-]\d{1,2}(?:[/-]\d{2,4})?\s*', 'DATE', text)  # Normalisasi tanggal
    text = re.sub(r'\s+', ' ', text)  # Hapus spasi berlebih
    text = re.sub(r'[^\w\s]', '', text)  # Hapus karakter non-alfanumerik
    text = ''.join([c for c in text if c not in string.punctuation])  # Hapus tanda baca
    text = ' '.join([w for w in text.split() if w not in custom_stop_words])  # Stopword removal
    text = ' '.join(stem_tokens(text.split()))  # Stemming
    return text.strip()

In [43]:
def preprocess_date_alodokter(text):
    text = str(text).replace("Terakhir diperbarui:", "").strip()
    try:
        # Set locale ke Indonesia
        locale.setlocale(locale.LC_TIME, "id_ID.UTF-8")
        dt = datetime.strptime(text, "%d %B %Y")
        return dt.strftime("%d %B %Y")
    except Exception:
        raise ValueError(f"Format tanggal tidak dikenali: {text}")

In [44]:
alodokter_df['title'] = alodokter_df['title'].apply(lambda x: preprocess_title_desc(x))
alodokter_df['description'] = alodokter_df['description'].apply(lambda x: preprocess_title_desc(x))
alodokter_df['content'] = alodokter_df['content'].apply(lambda x: preprocess_content_alodokter(x))
alodokter_df['date'] = alodokter_df['date'].apply(lambda x: preprocess_date_alodokter(x))
alodokter_df['tag'] = alodokter_df['tag'].str.lower()

In [45]:
alodokter_df.head()

,title,tag,link,date,description,content,source
0,overthinking kenal ciri sebab dampak atas,hidup sehat,https://www.alodokter.com/overthinking-kenali-...,16 September 2025,overthinking biasa pikir pikir lelah terusmene...,overthinking biasa pikir pikir lelah terusmene...,alodokter
1,trauma healing bantu pulih dari luka batin,kesehatan,https://www.alodokter.com/trauma-healing-cara-...,16 September 2025,trauma healing proses bantu pulih dari luka ba...,trauma healing proses bantu pulih dari luka ba...,alodokter
2,mansplaining perilaku rendah sadar,kesehatan,https://www.alodokter.com/mansplaining-perilak...,16 September 2025,mansplaining pola komunikasi lakilaki rendah guru,mansplaining pola komunikasi lakilaki rendah g...,alodokter
3,fungsi otak kiri kunci mampu analitis matematika,kesehatan,https://www.alodokter.com/fungsi-otak-kiri-kun...,16 September 2025,fungsi otak kiri pegang peran mampu pikir logi...,fungsi otak kiri pegang peran mampu pikir logi...,alodokter
4,daun jati belanda kenal manfaat sehat,kesehatan,https://www.alodokter.com/daun-jati-belanda-ke...,16 September 2025,daun jati belanda herbal tradisional percaya b...,daun jati belanda herbal tradisional percaya b...,alodokter


# Detik Health

In [46]:
detik_df = pd.read_csv('../artikel/detik_health.csv', sep=';')
detik_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   title        100 non-null    object 
 1   tag          0 non-null      float64
 2   link         100 non-null    object 
 3   date         100 non-null    object 
 4   description  100 non-null    object 
 5   content      100 non-null    object 
dtypes: float64(1), object(5)
memory usage: 4.8+ KB


In [47]:
detik_df['source'] = 'detik'

In [48]:
detik_df['tag'] = detik_df['tag'].fillna('Kesehatan Umum')

In [49]:
detik_df.head()

,title,tag,link,date,description,content,source
0,Klaim BPJS Kesehatan untuk Gangguan Jiwa Naik!...,Kesehatan Umum,https://health.detik.com/berita-detikhealth/d-...,2025-09-16 10:22:44,Direktur Utama BPJS Kesehatan Ali Ghufron Mukt...,Direktur Utama BPJS Kesehatan Ali Ghufron Mukt...,detik
1,"Catat Rekor Baru, Pria Ini Positif COVID-19 Se...",Kesehatan Umum,https://health.detik.com/berita-detikhealth/d-...,2025-09-16 09:22:47,Seorang pria dengan sistem kekebalan tubuh yan...,Seorang pria dengan sistem kekebalan tubuh yan...,detik
2,Strategi BPOM RI Berantas Produk Obat dan Maka...,Kesehatan Umum,https://health.detik.com/berita-detikhealth/d-...,2025-09-16 09:22:49,Badan Pengawas Obat dan Makanan (BPOM RI) berg...,Badan Pengawas Obat dan Makanan (BPOM RI) berg...,detik
3,"Curhat Pria Lahir dari Donor Sperma, Bingung C...",Kesehatan Umum,https://health.detik.com/berita-detikhealth/d-...,2025-09-16 08:22:52,Seorang pria di Inggris merupakan salah satu a...,Seorang pria di Inggris merupakan salah satu a...,detik
4,"Pastikan Indomie Aman, Indofood Soroti Beda Re...",Kesehatan Umum,https://health.detik.com/berita-detikhealth/d-...,2025-09-16 06:22:54,Laporan kandungan etilen oksida (EtO) dalam In...,Laporan kandungan etilen oksida (EtO) dalam In...,detik


In [50]:
def preprocess_date_detik(text):
    text = text.strip().strip("'")
    
    dt = None
    try:
        dt = datetime.strptime(text, "%Y-%m-%d %H:%M:%S")
    except ValueError:
        pass
    
    if dt is None:
        try:
            # set locale ke Indonesia
            locale.setlocale(locale.LC_TIME, "id_ID.UTF-8")
            dt = datetime.strptime(text, "%A, %d %b %Y %H:%M")
        except Exception:
            raise ValueError(f"Format tanggal tidak dikenali: {text}")
    
    return dt.strftime("%d %B %Y")

In [51]:
def preprocess_content_detik(text):
    text = text.lower()
    text = normalize_text(text, istilah_kesehatan)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE) # Remove URLs
    text = re.sub(r'\@w+|\#','', text) # Remove mentions and hashtags
    text = re.sub(r'\s*\d{1,2}[/-]\d{1,2}(?:[/-]\d{2,4})?\s*', 'DATE', text) # Remove dates in parentheses
    text = re.sub(r'advertisement scroll to continue with content', '', text, flags=re.IGNORECASE) # Remove advertisement
    text = re.sub(r'\s+', ' ', text) # Remove extra whitespace
    text = re.sub(r'[^\w\s]', '', text) # Remove non-words and non-white spaces
    text = ''.join([char for char in text if char not in string.punctuation]) # Remove punctuations
    text = ' '.join([word for word in text.split() if word not in custom_stop_words]) # Stopwords removal
    text = ' '.join(stem_tokens(text.split()))

    return text

In [52]:
detik_df['title'] = detik_df['title'].apply(lambda x: preprocess_title_desc(x))
detik_df['description'] = detik_df['description'].apply(lambda x: preprocess_title_desc(x))
detik_df['content'] = detik_df['content'].apply(lambda x: preprocess_content_detik(x))
detik_df['date'] = detik_df['date'].apply(lambda x: preprocess_date_detik(x))

In [53]:
detik_df.head()

,title,tag,link,date,description,content,source
0,klaim bpjs sehat ganggu jiwa rp 2 triliun rinc...,Kesehatan Umum,https://health.detik.com/berita-detikhealth/d-...,16 September 2025,direktur utama bpjs sehat ali ghufron mukti ni...,direktur utama bpjs sehat ali ghufron mukti ni...,detik
1,catat rekor pria positif covid19 2,Kesehatan Umum,https://health.detik.com/berita-detikhealth/d-...,16 September 2025,pria sistem kebal tubuh lemah idap covid19 aku...,pria sistem kebal tubuh lemah idap covid19 aku...,detik
2,strategi bpom ri berantas produk obat makan ka...,Kesehatan Umum,https://health.detik.com/berita-detikhealth/d-...,16 September 2025,badan awas obat makan bpom ri gerak cepat bera...,badan awas obat makan bpom ri gerak cepat bera...,detik
3,curhat pria lahir dari donor sperma bingung ca...,Kesehatan Umum,https://health.detik.com/berita-detikhealth/d-...,16 September 2025,pria inggris salah anak lahir dari donor sperma,pria inggris salah anak lahir dari donor sperm...,detik
4,pasti indomie aman indofood sorot beda regulas...,Kesehatan Umum,https://health.detik.com/berita-detikhealth/d-...,16 September 2025,lapor kandung etilen oksida eto indomie varian...,lapor kandung etilen oksida eto indomie varian...,detik


# Kompas Health

In [54]:
kompas_df = pd.read_csv('../artikel/kompas_health.csv', sep=';')
kompas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        51 non-null     object
 1   tag          51 non-null     object
 2   link         51 non-null     object
 3   date         51 non-null     object
 4   description  51 non-null     object
 5   content      51 non-null     object
dtypes: object(6)
memory usage: 2.5+ KB


In [55]:
kompas_df['source'] = 'kompas'

In [56]:
kompas_df.head()

,title,tag,link,date,description,content,source
0,Mengapa Anak Harus Mendapatkan Vaksin Campak? ...,HEALTH,https://health.kompas.com/read/25I15161500668/...,"15/09/2025, 16:15 WIB",Jangan abaikan vaksin campak. Vaksin ini melin...,KOMPAS.com -Mengapa vaksin campak penting untu...,kompas
1,Cara Sederhana Mengatasi Brain Fog,HEALTH,https://health.kompas.com/read/25I15144420968/...,"15/09/2025, 14:44 WIB",Brain fog merujuk pada sekumpulan gejala kogni...,KOMPAS.com -Ketika kesibukan dan tingkat stres...,kompas
2,Pemprov Babel Segera Bangun Rumah Sakit Jantun...,HEALTH,https://health.kompas.com/read/25I14224517668/...,"14/09/2025, 22:45 WIB",Pemprov Kepulauan Babel rencanakan pembangunan...,KOMPAS.com- Pembangunan Rumah Sakit Jantung da...,kompas
3,"Kusta Jadi Masalah Serius di Papua Barat, Keme...",HEALTH,https://health.kompas.com/read/25I14222312268/...,"14/09/2025, 22:23 WIB",DPD RI desak Kemenkes bentuk tim investigasi k...,KOMPAS.com- Kasus kusta di Papua Barat masih m...,kompas
4,Modul Penanganan Psikosomatis Dokter Indonesia...,HEALTH,https://health.kompas.com/read/25I13184500268/...,"13/09/2025, 18:45 WIB","Sejak 2010, dr. Andri memfokuskan praktik klin...",KOMPAS.com -Dalam ajang bergengsi European Ac...,kompas


In [57]:
def preprocess_date_kompas(text):
    text = text.strip().replace(" WIB", "")  # hapus WIB
    
    try:
        dt = datetime.strptime(text, "%d/%m/%Y, %H:%M")
    except ValueError:
        raise ValueError(f"Format tanggal tidak dikenali: {text}")
    
    return dt.strftime("%d %B %Y")

In [58]:
def preprocess_content_kompas(text):
    text = text.lower()
    text = normalize_text(text, istilah_kesehatan)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE) # Remove URLs
    text = re.sub(r'\@w+|\#','', text) # Remove mentions and hashtags
    text = re.sub(r'kompas\.com\s*[–-]\s*', '', text, flags=re.MULTILINE) # Remove [NAMA KOTA], kompas.com -
    text = re.sub(r'baca juga:.*', '', text, flags=re.MULTILINE) # Remove baca juga
    text = re.sub(r'^.*\nartikel kompas\.id$', '', text, flags=re.MULTILINE) # Remove artikel kompas.id
    text = re.sub(r'\s*\d{1,2}[/-]\d{1,2}(?:[/-]\d{2,4})?\s*', 'DATE', text) # Remove dates in parentheses
    text = re.sub(r'\s+', ' ', text) # Remove extra whitespace
    text = re.sub(r'[^\w\s]', '', text) # Remove non-words and non-white spaces
    text = ''.join([char for char in text if char not in string.punctuation]) # Remove punctuations
    text = ' '.join([word for word in text.split() if word not in custom_stop_words]) # Stopwords removal
    text = ' '.join(stem_tokens(text.split()))

    return text

In [59]:
kompas_df['title'] = kompas_df['title'].apply(lambda x: preprocess_title_desc(x))
kompas_df['description'] = kompas_df['description'].apply(lambda x: preprocess_title_desc(x))
kompas_df['content'] = kompas_df['content'].apply(lambda x: preprocess_content_kompas(x))
kompas_df['date'] = kompas_df['date'].apply(lambda x: preprocess_date_kompas(x))

In [60]:
kompas_df.head()

,title,tag,link,date,description,content,source
0,anak vaksin campak jelas kemenkes,HEALTH,https://health.kompas.com/read/25I15161500668/...,15 September 2025,abai vaksin campak vaksin lindung anak dari sa...,vaksin campak anak menteri sehat kemenkes vaks...,kompas
1,sederhana atas brain fog,HEALTH,https://health.kompas.com/read/25I15144420968/...,15 September 2025,brain fog rujuk kumpul gejala kognitif sulit k...,sibuk tingkat stres tingkat aneh tibatiba masu...,kompas
2,pemprov babel bangun rumah sakit jantung stroke,HEALTH,https://health.kompas.com/read/25I14224517668/...,14 September 2025,pemprov pulau babel rencana bangun rs jantung ...,bangun rumah sakit jantung stroke provinsi pul...,kompas
3,kusta serius papua barat kemenkes bentuk tim i...,HEALTH,https://health.kompas.com/read/25I14222312268/...,14 September 2025,dpd ri desak kemenkes bentuk tim investigasi k...,kusta papua barat serius 2024 catat 796 orang ...,kompas
4,modul tangan psikosomatis dokter indonesia pre...,HEALTH,https://health.kompas.com/read/25I13184500268/...,13 September 2025,2010 dr andri fokus praktik klinis pasien gang...,ajang gengsi european academy of psychosomatic...,kompas


# Merging

In [61]:
# Menggabungkan semua dataframe
all_df = pd.concat([biofarma_df, klikdokter_df, halodoc_df, alodokter_df, detik_df, kompas_df], ignore_index=True)
all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504 entries, 0 to 503
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        504 non-null    object
 1   tag          504 non-null    object
 2   link         504 non-null    object
 3   date         504 non-null    object
 4   description  504 non-null    object
 5   content      504 non-null    object
 6   source       504 non-null    object
dtypes: object(7)
memory usage: 27.7+ KB


In [62]:
all_df.isnull().sum()

title          0
tag            0
link           0
date           0
description    0
content        0
source         0
dtype: int64

In [63]:
all_df.insert(0, 'article_index', range(1, len(all_df) + 1))
all_df.head()

,article_index,title,tag,link,date,description,content,source
0,1,5 rawat ginjal sehat cegah sakit ginjal,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/det...,3 September 2025,kenal rawat ginjal hindar dari gejala ginjal m...,ginjal organ vital fungsi nyaring darah buang ...,Biofarma
1,2,7 tips jaga sehat mata,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/det...,29 Agustus 2025,tips jaga sehat mata nutrisi gizi kacamata lin...,jaga sehat mata mata peran aktivitas seharihar...,Biofarma
2,3,sepele 10 sebab campak abai,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/det...,28 Agustus 2025,kenal faktor utama sebab campak gejala khas ce...,sebab campak campak sakit tular sebab virus ca...,Biofarma
3,4,campak sebab gejala cegah,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/det...,25 Agustus 2025,campak bahaya biar periksa dokter gejala muncu...,campak campak rubella sakit tular sebab virus ...,Biofarma
4,5,mudah jaga sehat gigi mulut hindar dari sakit,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/det...,20 Agustus 2025,sehat gigi mulut kondisi gigi gusi lidah rongg...,sehat gigi mulut sehat gigi mulut kondisi gigi...,Biofarma


In [64]:
all_df.to_csv('../artikel/all_health_articles.csv', index=False)